# Proyecto 1

In [37]:
# librería Natural Language Toolkit, usada para trabajar con textos 
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
# Descarga todas las palabras vacias, es decir, aquellas que no aportan nada al significado del texto

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Importaciones 

In [40]:
 # Instalación de librerias
import pandas as pd
import numpy as np
import sys
from markupsafe import escape
#from pandas_profiling import ProfileReport 

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

# Carga de datos

In [41]:
data=pd.read_csv('SuicidiosProyecto.csv', sep=',', encoding = 'utf-8')
data_s=data

In [42]:
data_s.head()

,Unnamed: 0,text,class
0,173271,i want to destroy myselffor once everything wa...,suicide
1,336321,I kinda got behind schedule with learning for ...,non-suicide
2,256637,I'm just not sure anymoreFirst and foremost: I...,suicide
3,303772,please give me a reason to liveThats too much ...,suicide
4,293747,27f struggling to find meaning moving forwardI...,suicide


In [43]:
data_s['class'].value_counts()

non-suicide    110165
suicide         85535
Name: class, dtype: int64

# Limpieza de datos

### Remover puntuacion

In [44]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
data_s['words']= data_s['text'].apply(lambda x:remove_punctuation(x))
data_s.head()

,Unnamed: 0,text,class,words
0,173271,i want to destroy myselffor once everything wa...,suicide,i want to destroy myselffor once everything wa...
1,336321,I kinda got behind schedule with learning for ...,non-suicide,I kinda got behind schedule with learning for ...
2,256637,I'm just not sure anymoreFirst and foremost: I...,suicide,Im just not sure anymoreFirst and foremost Im ...
3,303772,please give me a reason to liveThats too much ...,suicide,please give me a reason to liveThats too much ...
4,293747,27f struggling to find meaning moving forwardI...,suicide,27f struggling to find meaning moving forwardI...


### Transformar todo a minusculas

In [45]:
data_s['words']= data_s['words'].apply(lambda x: x.lower())
data_s.head()

,Unnamed: 0,text,class,words
0,173271,i want to destroy myselffor once everything wa...,suicide,i want to destroy myselffor once everything wa...
1,336321,I kinda got behind schedule with learning for ...,non-suicide,i kinda got behind schedule with learning for ...
2,256637,I'm just not sure anymoreFirst and foremost: I...,suicide,im just not sure anymorefirst and foremost im ...
3,303772,please give me a reason to liveThats too much ...,suicide,please give me a reason to livethats too much ...
4,293747,27f struggling to find meaning moving forwardI...,suicide,27f struggling to find meaning moving forwardi...


### Reeplazar números

In [46]:
'''def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    
    if text.isdigit():
        new_word = p.number_to_words(text)
        new_words.append(new_word)
    else:
        new_words.append(text)
    return new_words
 
data_s['words']= data_s['words'].apply(lambda x: replace_numbers(x))
data.head()'''

'def replace_numbers(text):\n    """Replace all interger occurrences in list of tokenized words with textual representation"""\n    p = inflect.engine()\n    new_words = []\n    \n    if text.isdigit():\n        new_word = p.number_to_words(text)\n        new_words.append(new_word)\n    else:\n        new_words.append(text)\n    return new_words\n \ndata_s[\'words\']= data_s[\'words\'].apply(lambda x: replace_numbers(x))\ndata.head()'

### Tokenizar

In [47]:
#defining function for tokenization
import re
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
#applying function to the column
data_s['words']= data_s['words'].apply(lambda x: tokenization(x))
data_s.head()

,Unnamed: 0,text,class,words
0,173271,i want to destroy myselffor once everything wa...,suicide,"[i, want, to, destroy, myselffor, once, everyt..."
1,336321,I kinda got behind schedule with learning for ...,non-suicide,"[i, kinda, got, behind, schedule, with, learni..."
2,256637,I'm just not sure anymoreFirst and foremost: I...,suicide,"[im, just, not, sure, anymorefirst, and, forem..."
3,303772,please give me a reason to liveThats too much ...,suicide,"[please, give, me, a, reason, to, livethats, t..."
4,293747,27f struggling to find meaning moving forwardI...,suicide,"[27f, struggling, to, find, meaning, moving, f..."


### Remover stopwords

In [49]:
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
data_s['words']= data_s['words'].apply(lambda x: remove_stopwords(x))
data_s.head()

,Unnamed: 0,text,class,words
0,173271,i want to destroy myselffor once everything wa...,suicide,"[want, destroy, myselffor, everything, startin..."
1,336321,I kinda got behind schedule with learning for ...,non-suicide,"[kinda, got, behind, schedule, learning, next,..."
2,256637,I'm just not sure anymoreFirst and foremost: I...,suicide,"[im, sure, anymorefirst, foremost, im, brazil,..."
3,303772,please give me a reason to liveThats too much ...,suicide,"[please, give, reason, livethats, much, dont, ..."
4,293747,27f struggling to find meaning moving forwardI...,suicide,"[27f, struggling, find, meaning, moving, forwa..."


### Remover non-ASCII

In [50]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words
data_s['words']= data_s['words'].apply(lambda x: remove_non_ascii(x))
data_s.head()

,Unnamed: 0,text,class,words
0,173271,i want to destroy myselffor once everything wa...,suicide,"[want, destroy, myselffor, everything, startin..."
1,336321,I kinda got behind schedule with learning for ...,non-suicide,"[kinda, got, behind, schedule, learning, next,..."
2,256637,I'm just not sure anymoreFirst and foremost: I...,suicide,"[im, sure, anymorefirst, foremost, im, brazil,..."
3,303772,please give me a reason to liveThats too much ...,suicide,"[please, give, reason, livethats, much, dont, ..."
4,293747,27f struggling to find meaning moving forwardI...,suicide,"[27f, struggling, find, meaning, moving, forwa..."


# Normalización